In [4]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [5]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [6]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [10]:
from openai import OpenAI

client = OpenAI()

#### Sending generic message to check structure of response returned

In [11]:
messages = [
    {
        "role": "user",
        "content": "Hi!"
    }
]

response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
    functions=functions
)

In [ ]:
response

<OpenAIObject chat.completion id=chatcmpl-9u1ySB26VH8AsqobjnbwMkXplR2TI at 0x1069ce0c0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello! How can I assist you today?",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1723140984,
  "id": "chatcmpl-9u1ySB26VH8AsqobjnbwMkXplR2TI",
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 76,
    "total_tokens": 86
  }
}

In [12]:
response_message = response.choices[0].message
response_message

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', function_call=None, tool_calls=None)

#### Asking weather information to check response structure for function call

In [13]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
    functions=functions
)

In [14]:
print(response)

ChatCompletion(id='chatcmpl-9u2696Z6xv6ZfVBefV5twpDBbOHov', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston","unit":"celsius"}', name='get_current_weather'), tool_calls=None))], created=1723141461, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=20, prompt_tokens=82, total_tokens=102))


In [15]:
response_message = response.choices[0].message

In [16]:
response_message.content

##### For generic message, if function is not used, then there is no `function_call` key in `response_message`

In [17]:
response_message.function_call

FunctionCall(arguments='{"location":"Boston","unit":"celsius"}', name='get_current_weather')

In [18]:
import json

args = json.loads(response_message.function_call.arguments)
function_name = response_message.function_call.name

print(f"Args: {args}\n\nFunction Name: {function_name}")

Args: {'location': 'Boston', 'unit': 'celsius'}

Function Name: get_current_weather


OpenAI does not call the function for us. It just tells which function to call. We have to manually call the function later

In [19]:
func = globals()[function_name]
output = func(**args)

output

'{"location": "Boston", "temperature": "72", "unit": "celsius", "forecast": ["sunny", "windy"]}'

#### Now, we only got what function to call and what are its parameters, we need to pass it as messages to openai.chat_completions to get coherent and context aware response

In [21]:
# append response message which returns function call
messages.append(response_message)

# append the output of the function call
messages.append({
    "role": "function",
    "name": "get_current_weather",
    "content": output,
})

final_response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
    functions=functions
)

print(final_response)

ChatCompletion(id='chatcmpl-9u26g9IBD9vR0s8yTlDAwqdEFiZtH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current weather in Boston is 72°C with a forecast of sunny and windy conditions.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723141494, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=19, prompt_tokens=200, total_tokens=219))


In [22]:
final_response.choices[0].message.content

'The current weather in Boston is 72°C with a forecast of sunny and windy conditions.'